In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os
import sys
import argparse
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [5]:
df_train = pd.read_csv("dataset/train.csv")
df_test = pd.read_csv("dataset/test.csv") # Test dataset has no target, we'll need to do a validation split

print('Training Set Shape = {}'.format(df_train.shape))
print('Training Set Memory Usage = {:.2f} MB'.format(df_train.memory_usage().sum() / 1024**2))
print('Test Set Shape = {}'.format(df_test.shape))
print('Test Set Memory Usage = {:.2f} MB'.format(df_test.memory_usage().sum() / 1024**2))

Training Set Shape = (7613, 5)
Training Set Memory Usage = 0.29 MB
Test Set Shape = (3263, 4)
Test Set Memory Usage = 0.10 MB


In [9]:
text = df_train['text']
target = df_train['target']

X_train, X_val, y_train, y_val = train_test_split(text, target, test_size=0.2, random_state=42)

In [17]:
from tensorflow.keras.layers import Layer
import tensorflow_hub as hub
from tensorflow.keras import backend as K

# class ElmoEmbeddingLayer(Layer):
#     def __init__(self, **kwargs):
#         self.dimensions = 1024
#         self.trainable = True
#         super(ElmoEmbeddingLayer, self).__init__(**kwargs)
#     def build(self, input_shape):
#         self.elmo = tf.compat.v1.Module ('https://tfhub.dev/google/elmo/2', trainable=self.trainable, name="{}_module".format(self.name))
#         self.trainable_weights += K.tf.trainable_variables(scope="^{}_module/.*".format(self.name))
#         super(ElmoEmbeddingLayer, self).build(input_shape)
#     def call(self, x, mask=None):
#         result = self.elmo(K.squeeze(K.cast(x, tf.string), axis=1),
#                       as_dict=True,
#                       signature='default',
#                       )['default']
#         return result
#     def compute_mask(self, inputs, mask=None):
#         return K.not_equal(inputs, '--PAD--')
#     def compute_output_shape(self, input_shape):
#         return (input_shape[0], self.dimensions)
    


In [19]:
from tensorflow.keras.models import Model
import tensorflow_hub as hub
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.layers import Dropout
# from tensorflow.keras.layers import Lambda
# from tensorflow.keras.layers import Concatenate
# from tensorflow.keras.layers import LSTM
# from tensorflow.keras.layers import Bidirectional
# from tensorflow.keras.layers import BatchNormalization
# from tensorflow.keras.layers import SpatialDropout1D
# from tensorflow.keras.layers import Embedding
# from tensorflow.keras.layers import GlobalAveragePooling1D
# from tensorflow.keras.layers import GlobalMaxPooling1D
# from tensorflow.keras.optimizers import Adam

max_length = 100

input_text = layers.Input(shape=(1,), dtype=tf.string)
# embedding = ElmoEmbeddingLayer()(input_text)
# Create Elmo embedding layer
elmo = hub.KerasLayer("https://tfhub.dev/google/elmo/3", trainable=False, name='elmo_embedding')

# Example usage
input_text = Input(shape=(max_length,), dtype=tf.string)
embedding = elmo(input_text)

dense = layers.Dense(256, activation='relu')(embedding)
pred = layers.Dense(1, activation='sigmoid')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', "f1"])
model.summary()
# model.fit(X_train, 
#           y_train,
#           validation_data=(X_val, y_val),
#           epochs=5,
#           batch_size=32)

TypeError: Exception encountered when calling layer 'elmo_embedding' (type KerasLayer).

Binding inputs to tf.function failed due to `too many positional arguments`. Received args: (<KerasTensor shape=(None, 100), dtype=string, sparse=None, name=keras_tensor_4>,) and kwargs: {} for signature: () -> Dict[['lstm_outputs1', TensorSpec(shape=(None, None, 1024), dtype=tf.float32, name=None)], ['lstm_outputs2', TensorSpec(shape=(None, None, 1024), dtype=tf.float32, name=None)], ['word_emb', TensorSpec(shape=(None, None, 512), dtype=tf.float32, name=None)], ['elmo', TensorSpec(shape=(None, None, 1024), dtype=tf.float32, name=None)], ['sequence_len', TensorSpec(shape=(None,), dtype=tf.int32, name=None)], ['default', TensorSpec(shape=(None, 1024), dtype=tf.float32, name=None)]].
Fallback to flat signature also failed due to: pruned(text): expected argument #0(zero-based) to be a Tensor; got KerasTensor (<KerasTensor shape=(None, 100), dtype=string, sparse=None, name=keras_tensor_4>).

Call argu